In [3]:
import stable_get_html
stable_get_html.get_html('https://baidu.com')

<Response [200]>

In [4]:
import requests
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.62",
}

In [5]:
params = {
    "apikey":"827d6e453fef4ce390afe017b5a7db4b",
    'uom':'ft',
    'locale':'en_US',
    "currency":"USD",
    "condition":"used",
    # "condition":"new",
    "multiFacetedBoatTypeClass":'["sail"]',
    'sort':'price-asc',
    'variant':'0',
    'isMultiSearch':'true',
    'fields':'mappedURL,tags,date.created,date.indexed,make,model,condition,boat.boatType,boat.class,year,boat.specifications.dimensions.lengths.nominal,price.type.amount,price.discount,owner.logos,owner.id,owner.name,media.0,id,aliases.imt,aliases.yachtworld,mediaCount,status,isCurrentModel,isOemModel,location.city,location.normalizedCity,location.countrySubDivisionCode,location.isoSubDivisionCode,location.countryCode,location.postalCode,boat.attributes,boat.cpybLogo,salesRep.certs',
    'page':'1',
}
res = requests.get('https://api-gateway.boats.com/api-yachtworld/search', params=params, headers=headers)
#    ?apikey=827d6e453fef4ce390afe017b5a7db4b&multiFacetedBoatTypeClass=["sail"]&uom=ft&locale=en_US&currency=USD&sort=price-desc&page=1&variant=0&seoContentType=type&videoType=type&isMultiSearch=true&fields=mappedURL,tags,date.created,date.indexed,make,model,condition,boat.boatType,boat.class,year,boat.specifications.dimensions.lengths.nominal,price.type.amount,price.discount,owner.logos,owner.id,owner.name,media.0,id,aliases.imt,aliases.yachtworld,mediaCount,status,isCurrentModel,isOemModel,location.city,location.normalizedCity,location.countrySubDivisionCode,location.isoSubDivisionCode,location.countryCode,location.postalCode,boat.attributes,boat.cpybLogo,salesRep.certs', headers=headers)
result = res.json()

In [6]:
result.keys()
search_result = result['search']
search_result.keys()
count = search_result['count']
first_page = search_result['currentPage']
last_page = search_result['lastPage']
search_result['records'][0]


{'mappedURL': 'https://www.yachtworld.com/yacht/1988-test-monitoring-boat-keep-active-5815243/',
 'tags': ['exchangesite.adb',
  'exchangesite.adobe.saint',
  'exchangesite.boats',
  'exchangesite.boot24',
  'exchangesite.botenbank',
  'exchangesite.botentekoop',
  'exchangesite.btol',
  'exchangesite.pls',
  'exchangesite.yw',
  'portal.boats',
  'portal.boattrader',
  'portal.yachtworld',
  'portal.adb',
  'portal.botentekoop',
  'portal.botenbank',
  'portal.boot24'],
 'date': {'created': '2016-05-20T22:51:30Z',
  'indexed': '2023-03-21T14:20:33.508Z'},
 'make': 'Test',
 'model': 'MONITORING BOAT KEEP ACTIVE',
 'condition': 'used',
 'boat': {'boatType': 'sail',
  'class': 'sail-day',
  'specifications': {'dimensions': {'lengths': {'nominal': {'ft': 14.92,
      'm': 4.55}}}}},
 'year': 1988,
 'price': {'type': {'amount': {'AUD': 184.06,
    'CAD': 166.93,
    'CHF': 113.63,
    'CZK': 2676.05,
    'DKK': 850.51,
    'EUR': 114.15,
    'GBP': 100.25,
    'HUF': 43235.7,
    'INR': 10

In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm, trange

records = [dict() for _ in range(count)] 

def process_page(current_page):
    my_params = params.copy()
    my_params['page'] = str(current_page)
    # res = requests.get('https://api-gateway.boats.com/api-yachtworld/search', params=my_params, headers=headers)
    res = stable_get_html.get_html('https://api-gateway.boats.com/api-yachtworld/search', params=my_params)
    result = res.json()
    search_result = result['search']
    assert search_result['currentPage']==current_page
    assert search_result['currentPageSize']==15 or current_page==last_page
    index = 15*(current_page-1)
    records[index:index+search_result['currentPageSize']] = search_result['records']
    
    
with ThreadPoolExecutor(max_workers=64) as t:
    tasks = [t.submit(process_page, current_page) for current_page in range(first_page, last_page+1)]
    for future in tqdm(as_completed(tasks), total=len(tasks)):
        pass

100%|██████████| 666/666 [43:07<00:00,  3.89s/it] 


In [24]:
len(records), count
all(map(lambda x: x!={}, records))

False

In [18]:
def trans(record):
    result = {}
    try:
        result['url'] = record['mappedURL']
        result['new'] = (record['condition']=='new')
        if record.get('price') is not None:
            result['price'] = record['price']['type']['amount']['USD']
        if record.get('location') is not None:
            result['location'] = record['location']
    except Exception as e:
        print(record)
        # print(e)
        raise e
    return result
trans(search_result['records'][0])

{'url': 'https://www.yachtworld.com/yacht/1988-test-monitoring-boat-keep-active-5815243/',
 'new': False,
 'price': 124,
 'location': {'city': 'Brooklyn',
  'normalizedCity': 'Brooklyn',
  'countrySubDivisionCode': 'NY',
  'isoSubDivisionCode': 'US-NY',
  'countryCode': 'US',
  'postalCode': '11201'}}

In [20]:
records[0:30]

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {}]

In [19]:
results = list(map(trans, records))
results

{}


KeyError: 'mappedURL'

In [ ]:
new_boats_base_link = "https://www.yachtworld.com/boats-for-sale/condition-new/type-sail/sort-price:asc/"
old_boats_base_link = "https://www.yachtworld.com/boats-for-sale/condition-used/type-sail/sort-price:asc/"